In [ ]:
# Torch e CUDA
import torch
import gc
from torch.utils.data import Subset

# Transformers e Training
from transformers import (
    TextStreamer,
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
# Dataset e valutazione
from datasets import load_dataset, Dataset
from evaluate import load
import bitsandbytes as bnb
# Metriche di valutazione
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix
)

# Data manipulation
import pandas as pd
import numpy as np

# Sistema e utility
import os
from dotenv import load_dotenv
from pathlib import Path
from datetime import datetime

# Visualizzazione
import seaborn as sns
import matplotlib.pyplot as plt


prompt

In [ ]:
# model_name="mistralai/Mistral-7B-Instruct-v0.3"
model_name = "meta-llama/Llama-3.2-1B-Instruct"
if "meta" in model_name:
    prompt_template = (
        "### Instruction:\n"
        "You are an expert software developer and bug triaging specialist. Your task is to predict whether a bug "
        "will be resolved in LESS than 50 DAYS or MORE than 50 DAYS based on the provided bug details.\n\n"
        
        "- Output '0' if the bug will be resolved in LESS than 50 DAYS.\n"
        "- Output '1' if the bug will be resolved in MORE than 50 DAYS.\n\n"
        
        "Your response MUST be strictly either '0' or '1'. Do NOT include any additional text, explanations, formatting, symbols, or extra characters in your response.\n\n"

        "### Input:\n"
        "Source: {source}\n"
        "Product: {product}"
        "Short Description: {short_desc}\n"
        "Priority: {priority}\n"
        "Severity: {bug_severity}\n"
        #"Estimated resolution time: {days_resolution}\n\n" - questo potrebbe influenzare troppo il modello per la predizione

        "### Example Responses:\n"
        "Input: Source: KDE | Product: Payment System | Short Description: Critical security vulnerability found in authentication system | Priority: P1 | Severity: Critical\n"
        "Output: 0\n\n"
        "Input: Source: OpenOffice | Product: UI Module | Short Description: UI glitch affecting low-impact visual elements in settings panel | Priority: P3 | Severity: Minor\n"
        "Output: 1\n\n"

        "### Output: {label}\n"
    )
else:
    prompt_template = (
        "You are an expert software developer and bug triaging specialist. Your task is to predict whether a bug "
        "will be resolved in LESS than 50 DAYS or MORE than 50 DAYS based on the provided bug details.\n\n"
        
        "- Output '0' if the bug will be resolved in LESS than 50 DAYS.\n"
        "- Output '1' if the bug will be resolved in MORE than 50 DAYS.\n\n"
        
        "Your response MUST be strictly either '0' or '1'. Do NOT include any additional text, explanations, formatting, symbols, or extra characters in your response.\n\n"

        "## Input:\n"
        "Source: {source}\n"
        "Product: {product}"
        "Short Description: {short_desc}\n"
        "Priority: {priority}\n"
        "Severity: {bug_severity}\n"
        #"Estimated resolution time: {days_resolution}\n\n" - questo potrebbe influenzare troppo il modello per la predizione

        "## Example Responses:\n"
        "Input: Source: KDE | Product: Payment System | Short Description: Critical security vulnerability found in authentication system | Priority: P1 | Severity: Critical\n"
        "0\n\n"
        "Input: Source: OpenOffice | Product: UI Module | Short Description: UI glitch affecting low-impact visual elements in settings panel | Priority: P3 | Severity: Minor\n"
        "1\n\n"

        "## Output:"
        "{label}"
    )
num_val = "2000" #1000, 2000, 5000, 9000

caricamento del modello

In [ ]:
max_seq_length = 2048
dtype = torch.float16 #altrimenti None
load_in_4bit = True
seed = 3407
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
#model_name="meta-llama/Llama-3.1-8B-Instruct"
#model_name="mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name, token = hf_token)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)


In [ ]:
# Peft -  Parameter Efficient Fine Tuning
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # Layer fondamentali per catturare relazioni tra token
    # q_proj : "Query projection", v_proj: "Value projection", k_proj : "Key projection", o_proh: "output projection"
    #target_modules=['q_proj', 'v_proj', 'k_proj','o_proj','gate_proj','up_proj','down_proj','lm_head','embedded_layers']
    target_modules = ['q_proj', 'v_proj', 'gate_proj', 'up_proj', 'down_proj'] #forse lm_head non serve perchè generiamo solo un singolo token
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

formattazione del prompt con i dati del dataset

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Assicuriamoci di aggiungere il token EOS alla fine

def formatting_prompts(examples, include_label=True):
    texts = []
    for source, product, short_desc, priority, bug_severity, label in zip(
        examples["source"],examples["product"], examples["short_desc"], examples["priority"], examples["bug_severity"], examples["label"]
    ):
        if include_label:
            text = prompt_template.format(
                source=source,
                product=product,
                short_desc=short_desc,
                priority=priority,
                bug_severity=bug_severity,
                label=label,  # La label viene passata solo se include_label=True
            ) + EOS_TOKEN
        else:
            text = prompt_template.format(
                source=source,
                product=product, 
                short_desc=short_desc,
                priority=priority,
                bug_severity=bug_severity,
                label="",  #  Non passiamo la label
            ) + EOS_TOKEN
        
        texts.append(text)
    
    return {"text": texts}


# Caricamento dataset
dataset = load_dataset(
    "csv",
    data_files={
        "train": f"../dataset_completo/balanced_datasets/balanced_train_{num_val}.csv", 
        "test": f"../dataset_completo/balanced_datasets/balanced_test.csv", 
        "val": f"../dataset_completo/balanced_datasets/balanced_validation.csv" 
    },
)

# Formattiamo il dataset con il nuovo prompt
# Applichiamo la funzione al dataset
dataset["train"] = dataset["train"].map(lambda x: formatting_prompts(x, include_label=True), batched=True)
dataset["val"] = dataset["val"].map(lambda x: formatting_prompts(x, include_label=False), batched=True)  # 🚨 Label nascosta
#dataset["test"] = dataset["test"].map(lambda x: formatting_prompts(x, include_label=False), batched=True)  # 🚨 Label nascosta


dataset['train'][0]

Fine-tuning del modello

LoRA (Low-Rank Adaptation) è una tecnica di adattamento di modelli di linguaggio, come GPT o altri LLM (Large Language Models), che permette di fare il fine-tuning del modello con un numero ridotto di parametri, mantenendo la prestazione e riducendo il costo computazionale. La sua filosofia si basa sull'idea che solo una piccola parte della rete neurale necessiti di adattamenti per specializzarsi su un compito specifico, anziché dover modificare l'intero modello.
Concetto di base

In LoRA, il modello pre-addestrato non viene modificato nei suoi pesi originali, ma si aggiungono matrici di adattamento a basso rango. Queste matrici vengono apprese durante il fine-tuning, consentendo di adattare il comportamento del modello a nuovi dati senza dover riaddestrare completamente l'intero sistema.
Matematicamente:

Consideriamo una rete neurale come una sequenza di trasformazioni lineari, per esempio una mappatura tra uno strato X e Y tramite una matrice di pesi W:

Y = W X

In un contesto tradizionale di fine-tuning, andremmo a ottimizzare W, ovvero i pesi del modello. Invece, LoRA modifica l'approccio. Per adattare un modello con LoRA, l'idea è di decomporre il cambiamento nei pesi in un prodotto di due matrici di basso rango.

    Decomposizione del cambiamento nei pesi: Invece di aggiornare direttamente la matrice di pesi W, LoRA introduce due nuove matrici A e B, dove A ha dimensioni d × r (dove d è la dimensione dell'input, e r è un valore che indica il rango ridotto) e B ha dimensioni r × d (dove d è la dimensione dell'output). L'idea è che r è molto più piccolo di d, e quindi questo approccio riduce significativamente il numero di parametri da ottimizzare.

    Modifica della trasformazione: L'aggiornamento del modello tramite LoRA non avviene direttamente su W, ma sulla sua "perturbazione" tramite il prodotto delle matrici A e B:

Y = (W + A B) X

Dove:

    W è la matrice dei pesi originale del modello pre-addestrato.
    A B è l'aggiornamento a basso rango che viene appreso durante il fine-tuning.

    Ottimizzazione: Durante il fine-tuning, i pesi A e B vengono aggiornati, mentre W rimane fisso. L'ottimizzazione si concentra esclusivamente su questi due nuovi parametri, che sono molto più piccoli rispetto ai pesi originali.

    Proprietà di basso rango: L'idea di utilizzare matrici a basso rango è che molte modifiche necessarie per adattare un modello a un nuovo compito possono essere catturate tramite piccole modifiche che sono sufficientemente espresse in uno spazio di dimensioni ridotte. Questo riduce il numero totale di parametri e quindi le risorse computazionali richieste per l'addestramento.

Vantaggi:

    Efficienza: Non è necessario riaddestrare l'intero modello, solo le matrici A e B, che sono di dimensioni molto più piccole.
    Memoria: Poiché A e B sono di basso rango, richiedono meno memoria e meno spazio per essere memorizzate e aggiornate.
    Versatilità: Poiché W rimane fisso, il modello pre-addestrato può essere riutilizzato per più compiti con differenti adattamenti.

Riepilogo matematico:

    L'aggiornamento della rete avviene come Y = (W + A B) X, dove A e B sono appresi durante il fine-tuning, e il rango di A e B è molto più basso rispetto a quello di W.
    Il numero di parametri che devono essere ottimizzati è molto inferiore rispetto al caso tradizionale di fine-tuning dell'intero modello, il che consente di risparmiare risorse computazionali.

LoRA quindi permette di adattare i modelli in modo più efficiente e con meno parametri rispetto ai metodi tradizionali di fine-tuning, mantenendo alta la qualità e la prestazione nel nuovo compito.

In [ ]:
from evaluate import load
from trl import SFTTrainer, SFTConfig
# Carichiamo la metrica di accuracy
#accuracy_metric = load("accuracy")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # ✅ Fix padding issue
    tokenizer.padding_side = "right"
model.train() 
directory = f"{model_name}".split("/")[-1].strip()
# 🔹 Configurazione per l'addestramento (usando SFTConfig)
sft_config = SFTConfig(
    output_dir=f"{directory}_{num_val}_ft",
    max_seq_length=2048,
    dataset_text_field="text",  # Cambia se necessario
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,  #  Più epoche per adattare bene LoRA
    gradient_accumulation_steps=4,  #  Ridotto per aggiornamenti più frequenti
    evaluation_strategy="steps",  #  Valutazione più frequente
    eval_steps=100, 
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=3,  #  Evita troppi checkpoint
    learning_rate=5e-5,  #  Aumentato per migliorare adattamento
    lr_scheduler_type="cosine",  #  Cosine decay per convergenza più fluida
    warmup_ratio=0.05,  # Warmup ridotto per velocizzare training
    fp16=True,  #  Mantieni mixed precision
    logging_steps=50,  #  Meno logging per ridurre overhead
    metric_for_best_model="eval_loss",  # 👈 Assicura che il modello salvi in base alla Validation Loss
    greater_is_better=False  # 👈 Perché una loss minore è meglio
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    peft_config=peft_config,
    max_seq_length=2048,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=sft_config,
    packing= False,
)

# Avviamo il training!
trainer_stats = trainer.train()
eval_results = trainer.evaluate()
print(trainer_stats)
print(eval_results)

In [ ]:
import pandas as pd
from pathlib import Path

# Definiamo le metriche da salvare
training_results = {
    "Dataset Size": num_val,  # Numero di dati usati per il fine-tuning
    "Training Loss": trainer_stats.training_loss,  # Training Loss
    "Train Time (s)": trainer_stats.metrics["train_runtime"],  # Tempo di addestramento
    "Steps": trainer_stats.global_step,  # Numero di passi (steps)
    "Samples/sec": trainer_stats.metrics["train_samples_per_second"],  # Campioni al secondo
    "Steps/sec": trainer_stats.metrics["train_steps_per_second"],  # Passi al secondo
    "Validation Loss": eval_results.get("eval_loss", None),  # Valutazione della loss
}

# Definiamo il file di destinazione per i risultati
results_file = f"{model_name}_fine_tuned_on_{num_val}/training_comparison.csv"

# Converti il dizionario in un DataFrame
training_results_df = pd.DataFrame([training_results])  # Passiamo una lista contenente il dizionario

# Assicura che la cartella esista prima di salvare
Path(results_file).parent.mkdir(parents=True, exist_ok=True)

# Salva il DataFrame nel file CSV, sovrascrivendo se già esistente
training_results_df.to_csv(results_file, index=False)

# Mostra la tabella aggiornata
print(training_results_df)

 

In [ ]:
directory = f"{model_name}".split("/")[-1].strip().lower()

model.save_pretrained(f"./fine_tuned_model_{directory}_{num_val}" )
tokenizer.save_pretrained(f"./fine_tuned_model_{directory}_{num_val}" )